# Chapter 42: Migrating to Next.js

Migrating existing applications to Next.js represents one of the most common—and challenging—tasks facing development teams today. Whether moving from Create React App's single-page application model, Gatsby's static site architecture, or Vue's ecosystem, each migration path requires careful consideration of routing paradigms, data fetching patterns, and deployment implications. A successful migration preserves existing SEO rankings, maintains user session state, and minimizes downtime while unlocking Next.js's performance benefits.

By the end of this chapter, you'll master planning migration strategies from Create React App, transitioning Gatsby sites to Next.js with minimal content disruption, adapting Vue/Nuxt patterns to React's ecosystem, implementing incremental adoption for large codebases, handling routing migration with compatibility layers, and solving common challenges around SEO preservation and state management.

## 42.1 From React to Next.js

Convert a standard React application to Next.js by restructuring around the file-system router and adapting client-side patterns to hybrid rendering.

### Project Structure Migration

```typescript
// Before: Traditional React src/ structure
// src/
//   components/
//     Header.tsx
//     Footer.tsx
//   pages/
//     Home.tsx
//     About.tsx
//   App.tsx
//   index.tsx

// After: Next.js App Router structure
// app/
//   layout.tsx          (Root layout replaces App.tsx)
//   page.tsx            (Home route)
//   about/
//     page.tsx          (About route)
//   components/
//     Header.tsx        (Can remain Client Components)
//     Footer.tsx

// app/layout.tsx - Root layout replaces React's index.tsx + App.tsx
import type { Metadata } from 'next';
import { Inter } from 'next/font/google';
import './globals.css';

const inter = Inter({ subsets: ['latin'] });

export const metadata: Metadata = {
  title: 'My App',
  description: 'Description',
};

export default function RootLayout({
  children,
}: {
  children: React.ReactNode;
}) {
  return (
    <html lang="en">
      <body className={inter.className}>
        {/* Global providers moved here */}
        <Providers>
          {children}
        </Providers>
      </body>
    </html>
  );
}
```

### Client Component Migration

```typescript
// Before: src/pages/Home.tsx (React Router)
import { useEffect, useState } from 'react';
import { useParams } from 'react-router-dom';

export default function Home() {
  const { id } = useParams();
  const [data, setData] = useState(null);
  
  useEffect(() => {
    fetch(`/api/data/${id}`)
      .then(r => r.json())
      .then(setData);
  }, [id]);
  
  return <div>{/* content */}</div>;
}

// After: app/page.tsx (Next.js App Router)
// Option 1: Keep as Client Component (quick migration)
'use client';

import { useEffect, useState } from 'react';

export default function Home() {
  const [data, setData] = useState(null);
  
  useEffect(() => {
    // Route params accessed via hooks in Client Components
    const id = window.location.pathname.split('/').pop();
    fetch(`/api/data/${id}`)
      .then(r => r.json())
      .then(setData);
  }, []);
  
  return <div>{/* content */}</div>;
}

// Option 2: Migrate to Server Component (recommended)
// app/items/[id]/page.tsx
async function getData(id: string) {
  const res = await fetch(`https://api.example.com/data/${id}`, {
    cache: 'force-cache', // or 'no-store' for dynamic
  });
  return res.json();
}

export default async function ItemPage({ 
  params 
}: { 
  params: { id: string } 
}) {
  const data = await getData(params.id);
  
  return <div>{/* content */}</div>;
}
```

## 42.2 From Create React App (CRA)

Migrate the most common React setup to Next.js with specific handling for react-scripts configurations and client-side dependencies.

### Automated Migration Script

```bash
# 1. Install Next.js alongside existing CRA
npm install next@latest react@latest react-dom@latest

# 2. Update package.json scripts
# Remove: "start": "react-scripts start"
# Replace with:
{
  "scripts": {
    "dev": "next dev",
    "build": "next build",
    "start": "next start"
  }
}

# 3. Move public/ assets (Next.js uses public/ similarly)
# CRA public/ -> Next.js public/ (no changes needed)

# 4. Migrate environment variables
# REACT_APP_ -> NEXT_PUBLIC_ (for client-side)
# Keep without prefix for server-side only
```

### Configuration Mapping

```javascript
// Before: craco.config.js or react-scripts overrides
module.exports = {
  webpack: {
    alias: {
      '@components': path.resolve(__dirname, 'src/components'),
    },
  },
};

// After: next.config.js
/** @type {import('next').NextConfig} */
const nextConfig = {
  // webpack config carries over directly
  webpack: (config) => {
    config.resolve.alias = {
      ...config.resolve.alias,
      '@components': path.join(__dirname, 'src/components'),
    };
    return config;
  },
  
  // Handle CRA's proxy config
  async rewrites() {
    return [
      {
        source: '/api/:path*',
        destination: 'http://localhost:3001/api/:path*', // Your backend
      },
    ];
  },
  
  // Transpile modules that were handled by CRA
  transpilePackages: ['some-esm-package'],
};

module.exports = nextConfig;
```

### Handling CRA-Specific Patterns

```typescript
// Before: CRA's service worker registration
// src/serviceWorkerRegistration.ts
// src/service-worker.ts

// After: Next.js with next-pwa
// next.config.js
const withPWA = require('next-pwa')({
  dest: 'public',
  register: true,
  skipWaiting: true,
  disable: process.env.NODE_ENV === 'development',
});

module.exports = withPWA({ /* config */ });

// Before: CRA's public/index.html meta tags
// <meta name="description" content="...">

// After: Next.js Metadata API
// app/layout.tsx or specific pages
import type { Metadata } from 'next';

export const metadata: Metadata = {
  description: '...',
  themeColor: '#000000',
  manifest: '/manifest.json',
};
```

## 42.3 From Gatsby

Migrate static sites and GraphQL-dependent architectures to Next.js while preserving build-time data fetching capabilities.

### Data Fetching Migration

```typescript
// Before: Gatsby's GraphQL page query
// src/pages/blog/{mdx.frontmatter__slug}.tsx
import { graphql } from 'gatsby';

export const query = graphql`
  query BlogPost($id: String!) {
    mdx(id: { eq: $id }) {
      frontmatter {
        title
        date
      }
      body
    }
  }
`;

export default function BlogPost({ data }) {
  return <article>{data.mdx.body}</article>;
}

// After: Next.js generateStaticParams + Server Components
// app/blog/[slug]/page.tsx
import { getBlogPosts, getBlogPost } from '@/lib/blog';

// Generate static paths (replaces Gatsby's file system route API)
export async function generateStaticParams() {
  const posts = await getBlogPosts();
  return posts.map((post) => ({
    slug: post.slug,
  }));
}

// Fetch data at build time (replaces GraphQL at build time)
async function getBlogPostData(slug: string) {
  return await getBlogPost(slug);
}

export default async function BlogPost({ 
  params 
}: { 
  params: { slug: string } 
}) {
  const post = await getBlogPostData(params.slug);
  
  return (
    <article>
      <h1>{post.title}</h1>
      <time>{post.date}</time>
      <div dangerouslySetInnerHTML={{ __html: post.content }} />
    </article>
  );
}

// Generate metadata (replaces Gatsby's helmet/head)
export async function generateMetadata({ params }: { params: { slug: string } }) {
  const post = await getBlogPostData(params.slug);
  return {
    title: post.title,
    description: post.excerpt,
  };
}
```

### Gatsby Config to Next.js Config

```javascript
// Before: gatsby-config.js
module.exports = {
  siteMetadata: {
    title: 'My Site',
    siteUrl: 'https://example.com',
  },
  plugins: [
    'gatsby-plugin-image',
    'gatsby-plugin-sharp',
    {
      resolve: 'gatsby-source-filesystem',
      options: {
        name: 'blog',
        path: `${__dirname}/blog/`,
      },
    },
  ],
};

// After: next.config.js + lib/site-config.ts
/** @type {import('next').NextConfig} */
const nextConfig = {
  images: {
    formats: ['image/webp', 'image/avif'], // Replaces gatsby-plugin-sharp
    remotePatterns: [{ hostname: 'cdn.example.com' }],
  },
  
  // Site metadata managed via Metadata API in layouts
};

// lib/site-config.ts - Centralized config (replaces siteMetadata)
export const siteConfig = {
  title: 'My Site',
  url: 'https://example.com',
  description: 'Site description',
} as const;
```

## 42.4 From Vue/Nuxt

Translate Vue patterns to React/Next.js for teams switching ecosystems, focusing on composables vs hooks and template syntax.

### Pattern Translation Guide

```vue
<!-- Before: Vue 3 Composition API -->
<!-- pages/index.vue -->
<script setup>
const { data, pending } = await useFetch('/api/posts');

const sortedPosts = computed(() => 
  data.value?.sort((a, b) => b.date - a.date)
);
</script>

<template>
  <div>
    <div v-if="pending">Loading...</div>
    <ul v-else>
      <li v-for="post in sortedPosts" :key="post.id">
        {{ post.title }}
      </li>
    </ul>
  </div>
</template>
```

```typescript
// After: Next.js App Router
// app/page.tsx (Server Component by default - no loading state needed)
import { getPosts } from '@/lib/api';

export default async function HomePage() {
  const posts = await getPosts();
  const sortedPosts = posts.sort((a, b) => 
    new Date(b.date).getTime() - new Date(a.date).getTime()
  );
  
  return (
    <ul>
      {sortedPosts.map((post) => (
        <li key={post.id}>{post.title}</li>
      ))}
    </ul>
  );
}

// Or with loading state (Client Component)
// app/page.tsx
import { Suspense } from 'react';
import { PostsList } from './posts-list';
import { PostsSkeleton } from './posts-skeleton';

export default function HomePage() {
  return (
    <Suspense fallback={<PostsSkeleton />}>
      <PostsList />
    </Suspense>
  );
}

// app/posts-list.tsx
'use client';

import useSWR from 'swr';

export function PostsList() {
  const { data: posts } = useSWR('/api/posts', fetcher);
  
  // Computed equivalent: useMemo
  const sortedPosts = useMemo(() => 
    posts?.sort((a, b) => new Date(b.date).getTime() - new Date(a.date).getTime()),
    [posts]
  );
  
  if (!posts) return <div>Loading...</div>;
  
  return (
    <ul>
      {sortedPosts?.map((post) => (
        <li key={post.id}>{post.title}</li>
      ))}
    </ul>
  );
}
```

### Nuxt Plugin to Next.js

```typescript
// Before: Nuxt plugin (plugins/api.ts)
export default defineNuxtPlugin((nuxtApp) => {
  const api = $fetch.create({
    baseURL: useRuntimeConfig().public.apiBase,
    onRequest({ options }) {
      const token = useCookie('token').value;
      options.headers.set('Authorization', `Bearer ${token}`);
    },
  });
  
  return {
    provide: { api },
  };
});

// Usage: const { $api } = useNuxtApp();

// After: Next.js fetcher with cookies
// lib/api/client.ts
import { cookies } from 'next/headers';

export async function fetchWithAuth(url: string, options?: RequestInit) {
  const cookieStore = cookies();
  const token = cookieStore.get('token')?.value;
  
  return fetch(`${process.env.NEXT_PUBLIC_API_BASE}${url}`, {
    ...options,
    headers: {
      ...options?.headers,
      'Authorization': token ? `Bearer ${token}` : '',
    },
  });
}

// lib/api/hooks.ts (Client-side equivalent)
'use client';

import useSWR from 'swr';
import { useCookies } from 'react-cookie';

export function useApi() {
  const [cookies] = useCookies(['token']);
  
  const fetcher = (url: string) => 
    fetch(url, {
      headers: {
        'Authorization': `Bearer ${cookies.token}`,
      },
    }).then(r => r.json());
  
  return { fetcher };
}
```

## 42.5 Migration Strategies

Choose between big-bang rewrites, strangler fig patterns, or parallel implementations based on application size and risk tolerance.

### Strangler Fig Pattern

```typescript
// app/[[...catchall]]/page.tsx
// Gradually migrate routes while keeping old app running

import { notFound } from 'next/navigation';

// List of migrated routes
const migratedRoutes = [
  '/about',
  '/products',
  '/blog',
];

export default function CatchAllPage({ 
  params 
}: { 
  params: { catchall?: string[] } 
}) {
  const path = '/' + (params.catchall?.join('/') || '');
  
  // If route is migrated, render Next.js version
  if (migratedRoutes.includes(path)) {
    // Render specific component based on path
    return <MigratedPage path={path} />;
  }
  
  // Otherwise, proxy to old application
  // This requires middleware or rewrites to handle
  notFound(); // Will trigger 404, handled by middleware below
}

// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

const migratedRoutes = ['/about', '/products', '/blog'];

export function middleware(request: NextRequest) {
  const { pathname } = request.nextUrl;
  
  // Check if route is migrated
  const isMigrated = migratedRoutes.some(route => 
    pathname.startsWith(route)
  );
  
  if (isMigrated) {
    return NextResponse.next(); // Let Next.js handle it
  }
  
  // Proxy to legacy app (e.g., running on different port/domain)
  const legacyUrl = new URL(pathname, 'https://legacy.example.com');
  return NextResponse.rewrite(legacyUrl);
}

export const config = {
  matcher: ['/((?!api|_next/static|_next/image|favicon.ico).*)'],
};
```

## 42.6 Incremental Migration

Migrate feature-by-feature while maintaining a functional application using Next.js's custom server or rewrites.

### Page-by-Page Migration

```typescript
// Step 1: Set up Next.js alongside existing app
// pages/index.tsx (Pages Router for gradual adoption)

// Step 2: Create compatibility layer for shared state
// lib/legacy-bridge.ts
export function syncWithLegacyApp(data: any) {
  // PostMessage to old React app
  if (window.parent !== window) {
    window.parent.postMessage({ type: 'NEXT_JS_UPDATE', data }, '*');
  }
  
  // Or use localStorage for cross-tab sync
  localStorage.setItem('nextjs-migration-state', JSON.stringify({
    timestamp: Date.now(),
    data,
  }));
}

// Step 3: Migrate one route at a time
// Before: React Router route /dashboard
// After: pages/dashboard.tsx

import { useEffect } from 'react';
import { useRouter } from 'next/router';

export default function Dashboard() {
  const router = useRouter();
  
  useEffect(() => {
    // Notify legacy app of navigation
    syncWithLegacyApp({ route: '/dashboard' });
  }, []);
  
  // Component implementation
  return <div>Dashboard (Migrated to Next.js)</div>;
}

// Step 4: Update legacy app links to point to new routes gradually
// In legacy app:
// <a href="/dashboard"> -> <Link href="/dashboard">
```

## 42.7 Common Migration Challenges

Solve SEO preservation, client state hydration mismatches, and authentication continuity during transitions.

### SEO Preservation

```typescript
// lib/seo/migration.ts
// Maintain existing URL structure and redirects

// next.config.js
const nextConfig = {
  // Handle URL changes from old structure
  async redirects() {
    return [
      // Old: /post/123 -> New: /blog/123
      {
        source: '/post/:id',
        destination: '/blog/:id',
        permanent: true, // 301 redirect preserves SEO juice
      },
      // Handle trailing slashes
      {
        source: '/about/',
        destination: '/about',
        permanent: true,
      },
    ];
  },
  
  // Handle headers for SEO
  async headers() {
    return [
      {
        source: '/:path*',
        headers: [
          {
            key: 'X-Robots-Tag',
            value: 'index, follow',
          },
        ],
      },
    ];
  },
};

// pages/[...slug].tsx or app/[...slug]/page.tsx
// Handle legacy URL parameters
export async function generateMetadata({ 
  params, 
  searchParams 
}: { 
  params: { slug: string[] };
  searchParams: { [key: string]: string | string[] | undefined };
}) {
  // Support old query params like ?id=123
  const legacyId = searchParams.id;
  
  if (legacyId) {
    // Fetch content by legacy ID, redirect or canonicalize
    return {
      alternates: {
        canonical: `/new-path/${legacyId}`,
      },
      robots: {
        index: false, // Don't index duplicate content
        follow: true,
      },
    };
  }
  
  return {};
}
```

### Hydration Mismatch Solutions

```typescript
// Problem: Window object or random values cause hydration errors

// Before (causes error):
export default function Component() {
  const [width, setWidth] = useState(window.innerWidth); // Error: window not defined on server
  
  return <div>{width}</div>;
}

// Solution 1: useEffect for client-only data
export default function Component() {
  const [width, setWidth] = useState(0); // Default for server
  
  useEffect(() => {
    setWidth(window.innerWidth); // Only runs on client
  }, []);
  
  return <div>{width}</div>;
}

// Solution 2: Dynamic import with SSR disabled
import dynamic from 'next/dynamic';

const ClientOnlyChart = dynamic(() => import('./chart'), {
  ssr: false,
  loading: () => <div>Loading chart...</div>,
});

// Solution 3: suppressHydrationWarning for unavoidable mismatches (dates, etc)
export default function Timestamp() {
  return (
    <time suppressHydrationWarning>
      {new Date().toLocaleString()}
    </time>
  );
}
```

### Authentication Continuity

```typescript
// lib/auth/migration.ts
// Maintain auth state between old and new apps

// Middleware to sync tokens
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function middleware(request: NextRequest) {
  const response = NextResponse.next();
  
  // Check for legacy auth cookie
  const legacyToken = request.cookies.get('legacy_session')?.value;
  const newToken = request.cookies.get('next-auth.session-token')?.value;
  
  // If user has legacy token but not new one, validate and migrate
  if (legacyToken && !newToken) {
    // Validate legacy token
    const isValid = await validateLegacyToken(legacyToken);
    
    if (isValid) {
      // Set Next.js auth cookie
      response.cookies.set('next-auth.session-token', legacyToken, {
        httpOnly: true,
        secure: process.env.NODE_ENV === 'production',
        sameSite: 'lax',
      });
    }
  }
  
  return response;
}

// lib/auth/adapter.ts
// Adapter to support both old and new session formats
export async function getSession(req: NextRequest) {
  // Try new format first
  const newSession = await getNextAuthSession(req);
  if (newSession) return newSession;
  
  // Fall back to legacy format
  const legacySession = await getLegacySession(req);
  if (legacySession) {
    // Transform to new format
    return {
      user: {
        id: legacySession.user_id,
        email: legacySession.email,
      },
      expires: legacySession.expires_at,
    };
  }
  
  return null;
}
```

## Key Takeaways from Chapter 42

1. **CRA Migration**: Replace `react-scripts` with Next.js dependencies, move `src/index.js` logic to `app/layout.tsx`, and convert `BrowserRouter` routes to file-system based routing. Use `next.config.js` webpack configuration to maintain existing aliases and custom build configurations.

2. **Gatsby Migration**: Replace GraphQL page queries with `generateStaticParams` and Server Component data fetching. Migrate `gatsby-image` to `next/image`, and move SEO configuration from `react-helmet` to the Metadata API in layouts or page exports.

3. **Strangler Fig Pattern**: Implement catch-all routes with middleware to proxy unmigrated paths to legacy applications while serving migrated routes directly from Next.js. This enables zero-downtime incremental migration with URL consistency.

4. **State Synchronization**: Bridge client-side state between old and new applications using `postMessage` API for iframe-based integration, `localStorage` for cross-tab communication, or shared cookies for authentication tokens during the transition period.

5. **SEO Preservation**: Implement 301 redirects in `next.config.js` for any URL structure changes, use `generateMetadata` to maintain title/description parity, and add canonical tags pointing to preferred URLs when supporting duplicate legacy routes temporarily.

6. **Hydration Fixes**: Resolve server/client mismatches by initializing state to server-safe defaults (empty strings/arrays) and populating actual values in `useEffect`, or using `dynamic()` imports with `ssr: false` for browser-only components like charts or maps.

7. **Vue to React Translation**: Map Vue's `computed()` to React's `useMemo()`, `watch()` to `useEffect()`, and `useFetch()` to Server Components (for build-time) or SWR/React Query (for client-side). Replace Vue templates with JSX, converting `v-if` to ternary operators and `v-for` to `.map()`.

## Coming Up Next

**Chapter 43: Pages Router to App Router Migration**

With your application now running on Next.js, you may be looking to adopt the latest App Router architecture. In Chapter 43, we'll explore the nuanced differences between the Pages Router and App Router, strategies for incremental adoption within existing Next.js apps, migrating data fetching from `getServerSideProps` to Server Components, converting `_app.js` to `layout.tsx`, and handling the mental model shift from page-level to component-level data fetching.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../7. Tooling_and_workflow/41. typescript_advanced.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='43. pages_router_to_app_router.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
